# Join Data Using PostgreSQL and AWS #  
This notebook is an exercise in using [PostgreSQL](https://www.postgresql.org/) on an [AWS EC2](https://aws.amazon.com/ec2/) [Ubuntu](https://www.ubuntu.com/) instance to join data for cleaning.  
It also demonstrates using [sshtunnel](https://sshtunnel.readthedocs.io/en/latest/) and [SQLAlchemy](https://www.sqlalchemy.org/) to access SQL tables on the remote instance locally.

This is an exercise because the joins can be performed easily using [pandas](http://pandas.pydata.org/) and the data size does not necessitate a remote instance.

I wrote the necessary SQL schema and imported the data prior to running this notebook.

In [ ]:
import pandas as pd
from sshtunnel import SSHTunnelForwarder
from sqlalchemy import create_engine

import pickle

### Setup SSH tunnel ###

In [ ]:
# SSH config shortcut
SSH_CONFIG_SHORTCUT = INSTANCE

server = SSHTunnelForwarder(
    SSH_CONFIG_SHORTCUT,
    ssh_config_file=CONFIG_FILE,
    remote_bind_address=(ADDRESS)
)

server.start()

### Create SQLAlchemy engine to access SQL database on instance ###

In [ ]:
# Postgres username, password, and database name
POSTGRES_IP_ADDRESS = ADDRESS ## This is localhost because SSH tunnel is active
POSTGRES_PORT = str(server.local_bind_port)
POSTGRES_USERNAME = USERNAME    
POSTGRES_PASSWORD = PASSWORD 
POSTGRES_DBNAME = DB

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_IP_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)

### Perform join ###

In [ ]:
raw_df = pd.read_sql_query('''SELECT * 
                                FROM filings
                                    RIGHT JOIN committees
                                    ON filings.committee = committees.name
                                    JOIN committee_data
                                    ON committees.committeeid = committee_data.committeeid
                                    RIGHT JOIN cf_committee_data
                                    ON committee_data.fec_id = cf_committee_data.id;''', cnx)

pickle.dump(raw_df,open('../pickle_jar/raw_join.p','wb'))

In [ ]:
server.close()